# Scrapping from multiple pages

In [2]:
 #Importing the required libraries
import pandas as pd             
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

In [3]:
#Defining the required lists

Title = []                 
Info = []
Ratings = []
tv = []
Eps = []
Start = []
End = []

#Writing the loop or the for loop, loops the pages from (1-8)
#sends requests to the url to get page content
#BeautifulSoup arranges the page content
#Writing the classes and tag to get the required content  using soup.find_all

for i in range(1,9):                
    url =f"https://myanimelist.net/topanime.php?type=cm&limit={i}"
    requests.get(url)           
    page = requests.get(url)
    page
    page.text
    soup = BeautifulSoup(page.text)      
    
    
    for i in soup.find_all("div",class_="di-ib clearfix"):            
        Title.append(i.text.strip())                                             
        
    for i in soup.find_all("div",class_="information di-ib mt4"):
        Info.append(i.text.strip())
        
        
    for i in soup.find_all("div", class_ = "js-top-ranking-score-col di-ib al"):
        Ratings.append(i.text.strip())
        
d = {"Title":Title,"Info":Info,"Ratings":Ratings}
df = pd.DataFrame(d)

for i in df["Info"]:
    a = re.findall("(T.|Movie)",i)
        
    if a:
        tv.append(a[0])
    else:
        tv.append(np.nan)

    ep = re.findall("\((\d.\seps)",i)
    if ep:
        Eps.append(ep[0])
    else:
        Eps.append(np.nan)


    start = re.findall("(\w[A-Za-z].\s.\d.\d)\s-",i)
    if start:
           Start.append(start[0])
    else:
        Start.append(np.nan) 

    end =re.findall("-(\s\w.+)\s",i)
    if end:
          End.append(end[0])
    else:
        End.append(np.nan)
        
       
    
        
        
# d1 = {"Title":Title,"Tv":tv,"Ep":Eps,"SD":Start,"Ed":End,"Ratings":Ratings}
# df1 = pd.DataFrame(d1)   

In [4]:
d1 = {"Titles":Title,"Tv":tv,"Ep":Eps,"SD":Start,"Ed":End,"Ratings":Ratings}          #Creating a dict using Key value pairs
df1 = pd.DataFrame(d1)                                                               #creating a DataFrame

In [5]:
df1            #The first DataFrame df1

,Titles,Tv,Ep,SD,Ed,Ratings
0,Fullmetal Alchemist: Brotherhood,TV,64 eps,Apr 2009,Jul 2010,9.09
1,Steins;Gate,TV,24 eps,Apr 2011,Sep 2011,9.07
2,Gintama°,TV,51 eps,Apr 2015,Mar 2016,9.06
3,Shingeki no Kyojin Season 3 Part 2,TV,10 eps,Apr 2019,Jul 2019,9.05
4,Gintama: The Final,Movie,NaN,Jan 2021,Jan 2021,9.04
...,...,...,...,...,...,...
395,Ashita no Joe 2,TV,47 eps,Oct 1980,Aug 1981,8.72
396,86 Part 2,TV,12 eps,Oct 2021,Mar 2022,8.71
397,Shouwa Genroku Rakugo Shinjuu: Sukeroku Futata...,TV,12 eps,Jan 2017,Mar 2017,8.71
398,Mob Psycho 100 III,TV,12 eps,Oct 2022,Dec 2022,8.71


In [6]:
df1.shape

(400, 6)

# MultScrapping

In [ ]:
urls = []                               
titles = []
anime_data = []

#Using loop iterating through first link
#writing a regex pattern to find the link
#If link is found appending it to urls

for i in range(1, 9):  
    url = f"https://myanimelist.net/topanime.php?type=cm&limit={i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    
    for title_elem in soup.find_all("div", class_="di-ib clearfix"):
        titles.append(title_elem.text.strip())
    
    
    for url_elem in soup.find_all("a", class_="hoverinfo_trigger fl-l ml12 mr8"):
        url = re.findall("(https://.*)\"\sid", str(url_elem))             
        if url:
            urls.append(url[0])                                           

# List to store dictionaries of titles and genres
# anime_data = []
# Looping through each URL to get genres,source,demographic,producers information
# Iterating  each URL and its near index using enumerate


for idx, url in enumerate(urls):                                 
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract genres
    genres = []
    producers = []

    for genre_tag in soup.find_all('span', itemprop='genre'):
        genres.append(genre_tag.text)
        
    source = soup.find('span', string='Source:')
    if source:
        source_text = source.find_next('a').text.strip()
    else:
        source_text = 'Not specified'
        
        
    demographic_tag = soup.find('span', string='Demographic:')
    if demographic_tag:
        demographic_text = demographic_tag.find_next('a').text.strip()
    else:
        demographic_text = 'Not specified'
        
        
    
    for producer_tag in soup.find_all('span', string='Producers:'):
        producer_links = producer_tag.find_next_siblings('a')
        for link in producer_links:
            producers.append(link.text.strip())
        
    
#Appending the details through keys and values  
# Appending the data for each anime as a dictionary to anime_data
# Assuming titles and URLs are in the same order

    anime_data.append({                                       
       'title': titles[idx],                                  
        'genres': genres,
        'source': source_text,
        'demographic':demographic_text,
        'producers':producers
        
        
    })
    

In [ ]:
import pandas as pd                                                        # Creating lists for each column of the DataFrame
titles_list = [entry['title'] for entry in anime_data]
genres_list = [entry['genres'] for entry in anime_data]
source_list = [entry['source'] for entry in anime_data]
producers_list = [entry['producers'] for entry in anime_data]
demographic_list = [entry['demographic'] for entry in anime_data]


d = {"Titles":titles_list,"Genres":genres_list,"Source":source_list,"Demographic":demographic_list,"Producers":producers_list}
df2 = pd.DataFrame(d)

In [ ]:
df2

In [21]:

df2.shape

(400, 5)

In [22]:
df2.drop(columns="Titles",inplace =True)                     #Removing the Titles column from df2

In [23]:
df2.head(20)

,Genres,Source,Demographic,Producers
0,"[Action, Adventure, Drama, Fantasy, Military, ...",Manga,Shounen,"[Aniplex, Square Enix, Mainichi Broadcasting S..."
1,"[Drama, Sci-Fi, Suspense, Psychological, Time ...",Drama,Not specified,"[Frontier Works, Media Factory, Kadokawa Shote..."
2,"[Action, Comedy, Sci-Fi, Gag Humor, Historical...",Manga,Shounen,"[TV Tokyo, Aniplex, Dentsu]"
3,"[Action, Drama, Suspense, Gore, Military, Surv...",Manga,Shounen,"[Production I.G, Dentsu, Mainichi Broadcasting..."
4,"[Action, Comedy, Drama, Sci-Fi, Gag Humor, His...",Manga,Shounen,"[TV Tokyo, Aniplex, Dentsu, Bandai, Warner Bro..."
5,"[Action, Comedy, Sci-Fi, Gag Humor, Historical...",Manga,Shounen,"[TV Tokyo, Aniplex, Dentsu, Trinity Sound, Mir..."
6,"[Action, Adventure, Fantasy, Shounen]",Manga,Shounen,"[VAP, Nippon Television Network, Shueisha]"
7,"[Drama, Sci-Fi, Adult Cast, Military, Space]",Drama,Not specified,"[TV Tokyo, Kitty Films, Tokuma Shoten, Tokuma ..."
8,"[Action, Comedy, Sci-Fi, Gag Humor, Historical...",Manga,Shounen,"[TV Tokyo, Aniplex, Dentsu, Shueisha, Miracle ..."
9,"[Action, Adventure, Fantasy, Shounen]",Manga,Shounen,"[TV Tokyo, Aniplex, Dentsu, Shueisha]"


In [24]:
df_f =pd.concat([df1,df2], axis=1)                  #Concating the two DataFrames

In [28]:
df_f.head(1)

,Titles,Tv,Ep,SD,Ed,Ratings,Genres,Source,Demographic,Producers
0,Fullmetal Alchemist: Brotherhood,TV,64 eps,Apr 2009,Jul 2010,9.09,"[Action, Adventure, Drama, Fantasy, Military, ...",Manga,Shounen,"[Aniplex, Square Enix, Mainichi Broadcasting S..."


In [32]:
df_f.shape                #Shape of the DataFrame 

(400, 10)

In [27]:
df_f.to_csv("Raw_Data_anime.csv")